In [2]:
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox

    
# -------------------- #
# --- Classe Alien --- #
# -------------------- #
  
    
class Alien (object):
    def __init__(self):   #définition des variables globales de la classe
        self.id = None
        self.alive = True
        self.width = 40
        self.gap = 1
        self.pim = tk.PhotoImage(file='alien.gif')
     
    def install_in(self, canvas, x, y):   #création de l'alien
        self.id=canvas.create_image(x, y,image=self.pim, tags="image")
          

# --------------------- #
# --- Classe Flotte --- #
# --------------------- #


class Fleet(object):
    def __init__(self):   #définition des variables globales de la classe
        self.lines_alien = 5
        self.columns_alien = 5
        self.aliens_inner_gap = 20
        fleet_size = self.lines_alien * self.columns_alien
        self.aliens_fleet = []
    
    def install_in(self, canvas):   #création de la flotte d'alien
        self.canvas = canvas
        x = 50
        y = 50
        for i in range (self.lines_alien):   #début de la création de la matrice d'aliens
            for j in range (self.columns_alien):
                a=Alien()
                self.aliens_fleet.append(a)
                a.install_in(self.canvas, x, y)
                x = x + 90
            x = 50
            y = y + 70     
        
    def move(self, canvas):  # mouvement de déplacement de la matrice d'aliens
        self.canvas.move(self.aliens_fleet, self.dx, 0)
        
    def move_fleet(self):   #déplacement de la flotte
        self.alien = Alien()
        max_w = int(self.canvas.cget("width"))
        cpt=0
        cpt_y=0
        x1, y1, x2, y2 = self.canvas.bbox(self.aliens_fleet[0].id)   #limite minimale (à gauche) avant de repartir à gauche
        x3, y3, x4, y4 = self.canvas.bbox(self.aliens_fleet[24].id)  #limite maximale (à droite) avant de repartir à droite
      
        if x4+20 >int(self.canvas.cget("width")):   #vérification si x(flotte) > width pour revenir à gauche
            cpt=cpt+1
            self.aliens_inner_gap=-self.aliens_inner_gap
        if x1<20:   #vérification si x(flotte) < 20 pour repartir à droite
            cpt=cpt+1
            self.aliens_inner_gap=-self.aliens_inner_gap
          
        if y3+100>int(self.canvas.cget("height")):   #abaissage de la matrice en y avec un pas de 100
            cpt_y=cpt_y+1
            cpt=-1
        for a in self.aliens_fleet:   #parcours de la matrice d'aliens
            self.canvas.move(a.id,self.aliens_inner_gap,0)
            if cpt_y==0:
                if cpt>0:
                    for a in self.aliens_fleet:
                        self.canvas.move(a.id,0,10)
                    cpt=0
        if cpt_y>0 and cpt<0:   #si on est arrivée en bas de la fenêtre alors on a perdu 
            self. canvas.create_text(500, 400, text='YOU LOSE', fill='red', font= ('Broadway',50)) #message quand on a perdu
            return   #la flotte s'arrête
        self.canvas.after(200, self.move_fleet)
    
    def start_animation(self):   #fonction pour activer l'animation précédente
        self.canvas.after(50, self.move_fleet)
    
                        
# --------------------- #
# --- Classe Bullet --- #
# --------------------- #    


class Bullet(object):
    def __init__(self, shooter):   #définition des variables globales de la classe
        self.radius = -20
        self.color = "red"
        self.speed = 8
        self.id = None
        self.shooter = shooter
        self.laser = tk.PhotoImage(file='laser.png')
        self.laser = self.laser.subsample(12)

    def install_in(self, canvas):   #création de la balle
        self.canvas=canvas
        x1, y1, x2, y2 = self.canvas.bbox(self.shooter)
        self.id=canvas.create_image(x1+24, y1, image=self.laser, tags="image")
        
    def move_in(self):   #déplacement de la balle
        x1, y1, x2, y2 = self.canvas.bbox(self.id)
        self.canvas.move(self.id, 0, -self.speed)  
        self.canvas.after(30, self.move_in)
        
        
# ----------------------- #
# --- Classe Defender --- #
# ----------------------- #
            
    
class Defender(object):
    def __init__(self):   #définition des variables globales de la classe
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets = 8
        self.fired_bullets = []
        self.pim = tk.PhotoImage(file='defender.png')
        
    def install_in(self, canvas):   #création de la bullet
        self.canvas = canvas
        cwidth = int(self.canvas.cget("width"))
        cheight = int(self.canvas.cget("height"))
        self.lx = cwidth // 2
        self.ly = cheight - 30
        self.id=self.canvas.create_image(self.lx, self.ly, image=self.pim, tags="image")
    
    def get_move_delta(self):   #fonction pour récupérer la vitesse du defender
        return self.move_delta
    
    def move_in(self, canvas, dx):   #fonction pour déplacer le défender
        self.canvas = canvas
        limite = int(self.canvas.cget("width"))
        posX = self.canvas.bbox(self.id)[0]
        if dx < 0:  
            if posX>self.width/2 + 5:   #limite à gauche pour le defender
                canvas.move(self.id, dx, 0)
        else:  
            if posX+self.width < limite-self.width/2 - 37:   #limite à droite pour le defender
                self.canvas.move(self.id, dx, 0) 
                      
    def fire(self, canvas):   #fonction pour pouvoir tirer
        self.canvas = canvas 
        if (len(self.fired_bullets) < 8):
            b = Bullet(self.id)
            b.install_in(self.canvas)
            self.fired_bullets.append(b) #on ajoute une bullet à la liste
            b.move_in() 
            
            
# ------------------- #
# --- Classe Game --- #
# ------------------- #   

    
class Game(object):
    def __init__(self, frame):   #définition des variables globales de la classe
        self.defender = Defender()
        self.alien=Alien()
        self.fleet = Fleet()
        self.frame = frame
        self.width = 800
        self.height = 600
        self.bg = tk.PhotoImage(file='space.png')
        self.canvas = tk.Canvas(self.frame, width=self.width, height=self.height)
        self.canvas.create_image(405, 300, image=self.bg, tag='image')
        self.canvas.pack()
        
    def start_animation(self):   #animation des différents éléments
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
        self.fleet.start_animation()
        
    def actions(self, event):   #on bind les touches du clavier aux différentes actions
        if event.keysym == 'Left':
            self.defender.move_in(self.canvas, -self.defender.get_move_delta())
        elif event.keysym == 'Right':
            self.defender.move_in(self.canvas, self.defender.get_move_delta())
        elif event.keysym == 'space':
            self.defender.fire(self.canvas)

            
# ----------------------------- #
# --- Classe Space Invaders --- #
# ----------------------------- #

       
class SpaceInvaders(object):
    def __init__(self):   #définition des variables globales de la classe
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack()
        self.game = Game(self.frame)
    def start(self):   #début de toutes les animations du jeu
        self.game.start_animation()
        self.root.bind("<Key>", self.game.actions)
        self.root.mainloop()
              

sp = SpaceInvaders()
sp.start()